# 4. 작사가 인공지능 만들기

## Step 1. 데이터 다운로드 및 라이브러리 불러오기
---

```shell
$ mkdir -p ~/aiffel/lyricist/models
$ ln -s ~/data ~/aiffel/lyricist/data
```

In [1]:
import glob
import os
import re
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Step 2. 데이터 읽어오기
---

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 폴더의 txt 파일을 모두 읽어서 raw_corpus 에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:15])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion", "I can't get no relief Business men, they drink my wine", 'Plowman dig my earth', 'None were level on the mind', 'Nobody up at his word', 'Hey, hey No reason to get excited', 'The thief he kindly spoke', 'There are many here among us', 'Who feel that life is but a joke', "But, uh, but you and I, we've been through that", 'And this is not our fate', "So let us stop talkin' falsely now", "The hour's getting late, hey All along the watchtower"]


## Step 3. 데이터 정제
---

#### 먼저 15개의 문장을 확인해보겠습니다.

In [4]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너띄기
    if idx > 15: break   # 먼저 문장 15개 확인해보기
        
    print(sentence)

 There must be some kind of way outta here
Said the joker to the thief
There's too much confusion
I can't get no relief Business men, they drink my wine
Plowman dig my earth
None were level on the mind
Nobody up at his word
Hey, hey No reason to get excited
The thief he kindly spoke
There are many here among us
Who feel that life is but a joke
But, uh, but you and I, we've been through that
And this is not our fate
So let us stop talkin' falsely now
The hour's getting late, hey All along the watchtower
Princes kept the view


#### 함수를 통해 입력된 문장을 정제할 것입니다.
1. 소문자로 바꾸고, 양쪽 공백 지우기
2. 특수문자 양쪽에 공백넣기
3. 여러개의 공백은 하나의 공백으로 바꾸기
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸기
5. 다시 양쪽의 공백 지우기
6. 문장의 시작에는 ```<start>```, 끝에는 ```<end>``` 추가하기

#### 그리고 마지막으로 문장이 잘 정제되는지 확인해보겠습니다.

In [6]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


#### 위에서 만든 정제 함수를 이용해서 정제 데이터를 구축합니다.

In [7]:
corpus = []

for sentence in raw_corpus:
    # 공백인 문장은 건너뜁니다
    if len(sentence) == 0:
        continue
    
    # 정제한 데이터 셋을 저장합니다.
    preprocessed_sentence = preprocess_sentence(sentence)
    
    # 정제한 데이터가 공백인 경우는 제외합니다.
    if len(preprocessed_sentence) == 0:
        continue
    # 정제된 문장의 구성요소가 15개를 초과하는 경우는 제외하였습니다.
    elif len(preprocessed_sentence.split()) > 15:
        continue
    else:
        corpus.append(preprocessed_sentence)
        
# 정제된 데이터의 크기와 15개의 데이터를 먼저 확인해보겠습니다.
print('데이터 크기:', len(corpus))
corpus[:15]

데이터 크기: 156227


['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men , they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey , hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>',
 '<start> who feel that life is but a joke <end>',
 '<start> and this is not our fate <end>',
 '<start> so let us stop talkin falsely now <end>',
 '<start> the hour s getting late , hey all along the watchtower <end>',
 '<start> princes kept the view <end>']

#### 입력된 문장을 <u>**토큰화(Tokenize)**</u> 하겠습니다.

In [8]:
def tokenize(corpus):
    # 13000개의 단어를 기억할 수 있는 tokenizer를 만듭니다.
    # 13000개 단어에 포함되지 못한 단어는 '<unk>'로 바꾸겠습니다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞출 것입니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  62 271 ...   0   0   0]
 [  2 117   6 ...   0   0   0]
 [  2  62  17 ...   0   0   0]
 ...
 [  2  75  45 ...   3   0   0]
 [  2  49   5 ...   0   0   0]
 [  2  13 635 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7ff078c2a2d0>


#### 생성된 텐서의 3번째 행, 15번째 열까지만 출력해 보겠습니다.

In [9]:
print(tensor[:3, :15])

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 30: break

[[   2   62  271   27   94  546   20   86  743   90    3    0    0    0
     0]
 [   2  117    6 6269   10    6 2310    3    0    0    0    0    0    0
     0]
 [   2   62   17  102  184 2718    3    0    0    0    0    0    0    0
     0]]
1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : that
16 : t
17 : s
18 : on
19 : your
20 : of
21 : we
22 : .
23 : like
24 : m
25 : all
26 : is
27 : be
28 : for
29 : up
30 : with


## Step 4. 평가 데이터셋 분리
----

#### 훈련 데이터와 평가 데이터를 분리합니다.

In [10]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  62 271  27  94 546  20  86 743  90   3   0   0   0]
[ 62 271  27  94 546  20  86 743  90   3   0   0   0   0]


In [11]:
n_random_state = 35

enc_train, enc_val, dec_train, dec_val = train_test_split(
    src_input,
    tgt_input,
    test_size = 0.2,
    random_state = n_random_state)

In [12]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)


In [13]:
print(enc_train[1, :-1])
print(dec_train[1, :-1])

[   2  230   13   87 9974   76    3    0    0    0    0    0    0]
[ 230   13   87 9974   76    3    0    0    0    0    0    0    0]


In [14]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 생성합니다.
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [29]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [30]:
# 데이터셋에서 데이터 한 배치만 불러보겠습니다.
for enc_sample, dec_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어보겠습니다.
model(enc_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-3.60191450e-04,  2.23801297e-04,  1.14300819e-04, ...,
          4.95599350e-04,  1.21036814e-04,  5.46641066e-04],
        [-1.56331836e-04,  1.73110791e-04,  2.12264829e-04, ...,
          1.01504766e-03,  1.76648518e-05,  5.91539312e-04],
        [ 1.62272729e-04, -4.29857500e-05,  1.32690591e-03, ...,
          1.41716108e-03,  3.53180279e-04,  6.38290658e-04],
        ...,
        [ 8.24119488e-04, -3.78367887e-03, -6.95905765e-04, ...,
         -4.61831875e-03,  1.76911347e-03,  1.76695792e-03],
        [ 1.11357006e-03, -4.21933085e-03, -8.43754620e-04, ...,
         -5.28054638e-03,  1.94843661e-03,  1.81809650e-03],
        [ 1.40855229e-03, -4.63318033e-03, -9.46886488e-04, ...,
         -5.85185550e-03,  2.13021622e-03,  1.86953309e-03]],

       [[-3.60191450e-04,  2.23801297e-04,  1.14300819e-04, ...,
          4.95599350e-04,  1.21036814e-04,  5.46641066e-04],
        [-7.65532430e-04,  2.27603436e-04, -4

In [31]:
model.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  12289024  
_________________________________________________________________
lstm_6 (LSTM)                multiple                  8392704   
_________________________________________________________________
lstm_7 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_3 (Dense)              multiple                  12301025  
Total params: 41,375,457
Trainable params: 41,375,457
Non-trainable params: 0
_________________________________________________________________


In [32]:
n_train_epochs = 10

# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss = loss, optimizer = optimizer)
model.fit(dataset, epochs = n_train_epochs)

Epoch 1/10
488/488 [==============================] - 185s 373ms/step - loss: 3.8962
Epoch 2/10
488/488 [==============================] - 182s 373ms/step - loss: 2.9722
Epoch 3/10
488/488 [==============================] - 182s 373ms/step - loss: 2.7653
Epoch 4/10
488/488 [==============================] - 182s 372ms/step - loss: 2.6151
Epoch 5/10
488/488 [==============================] - 182s 373ms/step - loss: 2.4796
Epoch 6/10
488/488 [==============================] - 182s 373ms/step - loss: 2.3610
Epoch 7/10
488/488 [==============================] - 182s 372ms/step - loss: 2.2444
Epoch 8/10
488/488 [==============================] - 181s 371ms/step - loss: 2.1388
Epoch 9/10
488/488 [==============================] - 182s 373ms/step - loss: 2.0347
Epoch 10/10
488/488 [==============================] - 181s 371ms/step - loss: 1.9312


In [33]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 입력받은 init_sentence을 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [36]:
generate_text(model, tokenizer, init_sentence="<start> is not", max_len=20)

'<start> is not the only one that can be <end> '

## 회고
### 1. embedding size와 hidden size의 조정에 따른 성능 변화를 좀더 관찰해보고 싶습니다.
### 2. 최적의 파라미터를 찾을 수 있는 방법이 있는지 궁금합니다.
### 3. 영어가 아닌 한글 작사의 경우는 어떻게 할 수 있을지 궁금합니다.